# Consignes

Cet énoncé vous est destiné à tous les trois. Les deux problèmes qu'il comporte sont entièrement construits à partir des TP réalisés ensemble. Les deux exercices sont des classiques.
Vous avez, comme déjà dit, droit à toute la matière que vous souhaitez (chatbots, notes personnelles, corrigés des TPs). \
À la fin de l'examen, vous enverrez à l'adresse tpdeeplearning@gmail.com une archive .tar contenant un notebook par problème et par exercice (soit quatre notebooks au maximum).

Vous serez évalués sur les points suivants:

- La rédaction.\
Présentez votre approche. En particulier, si vous utilisez un chatbot (ce que je recommande), vous pouvez simplement copier les prompts que vous utilisez dans des cellules du notebook tout en mettant en avant votre esprit critique (par exemple, en commentant les réponses fournies par la machine). Dans tous les cas, notez que c'est la qualité de votre production qui sera jugée avant tout. Il n'est donc pas primordial d'avoir été jusqu'au bout de l'énoncé.

- La propreté des notebooks.\
Ils doivent être organisés et faciles à lire. Je dois pouvoir faire tourner le code sans problème et en comprendre les éléments. Il faut donc commenter un minimum.


# Problème n°1 (1h)

Dans ce problème, il s'agit de tester une parade à l'imperfection des résultats au TP n°5, partie 2.

- Refaire tourner le code solution du TP°5 (avec deux échantillons de bruit noise et noise2 additionnels) en stockant les poids du réseau au bout de 40 époques.

- Pouvez-vous rappeler le défaut qui avait été observé dans la question 2 ?
  Ré-expliquez son origine en terme de médiane conditionnelle.

- Nous avons évoqué plusieurs parades. La première consiste simplement à ajouter un nouvel échantillon de bruit (noise3). Faites une expérience sur quarante époques. Procédez ensuite à une comparaison qualitative avec les sorties du  modèle précédent. Faites enfin une comparaison quantitative (limitée) à partir des images ne contenant que du bruit.

- Nous avons aussi évoqué la régression quantile. Pouvez-vous expliquer en quoi cette forme de régression est adaptée ? Faites une expérience sur quarante époque en visant le quantile d'ordre 0.4 (dans la configuration initiale, c'est à dire avec deux échantillons de bruit seulement). Le problème est-il parfaitement résolu ? Comme ci-dessus, vérifier de manière qualitative et quantitative.


# Problème n°2 (1h)

 Dans le TP n°4 sur les transformers, nous avons appris à un modèle à préciser la parité d'une suite de zéros et de uns. Vous devez :
- reprendre le transformer proposé et le faire converger sur des suites de taille 64.
- définir un CNN pour la classification binaire dont le champ réceptif est proche de 32 comportant trois couches de convolution et deux couches complètement connectées.
- comparer les courbes d'apprentissage du CNN et du transformer (finesse sur le jeu de validation)
- comparer le nombre de poids dans chacun des modèles
- conclure

## Exercice n°1 (1/2 h)




Dans le modèle ci-dessous, pouvez-vous préciser :
- la taille du champ réceptif associé à un pixel d'une carte de caractéristique en sortie de enc2, par un calcul théorique.
- par une démarche empirique simple, la taille du champ réceptif associé à un pixel en sortie.


In [ ]:
import torch
import torch.nn as nn

class MyNN(nn.Module):
    def __init__(self, in_ch=3, out_ch=1, base=16):
        super().__init__()

        # 2 Conv2D
        self.enc1 = nn.Sequential(
            nn.Conv2d(in_ch, base, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(),
            nn.Conv2d(base, base, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(),
        )

        # 2 Conv2D, first with stride=2
        self.enc2 = nn.Sequential(
            nn.Conv2d(base, base * 2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.Conv2d(base * 2, base * 2, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(),
        )

        # again 2 Conv2D, first with stride=2
        self.enc3 = nn.Sequential(
            nn.Conv2d(base * 2, base * 4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.Conv2d(base * 4, base * 4, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(),
        )

        # alternance Conv2D / ConvTranspose2D
        self.mid = nn.Conv2d(base * 4, base * 4, kernel_size=3, padding=1, bias=False)

        self.up1 = nn.ConvTranspose2d(base * 4, base * 2, kernel_size=2, stride=2, bias=False)
        self.dec1 = nn.Conv2d(base * 2, base * 2, kernel_size=3, padding=1, bias=False)

        self.up2 = nn.ConvTranspose2d(base * 2, base, kernel_size=2, stride=2, bias=False)
        self.dec2 = nn.Conv2d(base, base, kernel_size=3, padding=1, bias=False)

        self.head = nn.Conv2d(base, out_ch, kernel_size=1, bias=False)

    def forward(self, x):
        x = self.enc1(x)
        x = self.enc2(x)
        x = self.enc3(x)

        x = torch.relu(self.mid(x))

        x = self.up1(x)
        x = torch.relu(self.dec1(x))

        x = self.up2(x)
        x = torch.relu(self.dec2(x))

        return self.head(x)

## Exercice n°2 (1/2 h)



- Charger le Visual Transformer (DeiT) hébergé sur Hugging Face à l'adresse :
"facebook/deit-tiny-patch16-224".
- Comment l'image est-elle transformée en séquence de vecteurs dans ce modèle ?
- Quelle est la taille de l'espace latent ?
- Quelle est la signification du premier token ?
- Classifier l'image fournie dans le TP n°2 (le chat) à l'aide de ce modèle (montrer les cinq premières classes choisies)
- Visualiser les scores attentionnels associés au premier token pour les trois premières têtes de la première couche attentionnelle. Certaines sont-elles interprétables ?